#### Packages for the analysis

In [2]:
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns

Checking what do we have at the current directory

In [5]:
os.listdir("./chinese-mnist")

['chinese_mnist.csv', '.DS_Store', 'chinese_mnist.tfrecords', 'data']

#### Loading the dataset

In [12]:
dataset_raw = pd.read_csv("./chinese-mnist/chinese_mnist.csv")

We have 15000 samples that each contains 5 features

In [17]:
dataset_raw.shape

(50, 5)

Let's see how does the dataset look

In [22]:
dataset_raw.sample(5).head()

,suite_id,sample_id,code,value,character
9151,22,10,4,3,三
12058,13,8,7,6,六
6664,69,4,1,0,零
2921,92,10,12,100,百
11095,17,5,6,5,五
